Trying spaceship titanic using ordinal encoding instead of one hot encoding

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score

In [3]:
dtest = pd.read_csv("train.csv")
dtest.info()

ftest = pd.read_csv("test.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
dtest.columns
dtest.drop("Name", axis=1)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [5]:
y = dtest.Transported
features = ['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

X = dtest[features]
X2 = ftest[features]

X_train, X_valid, y_train, y_valid = train_test_split(X,y)

In [12]:
from sklearn.preprocessing import OrdinalEncoder
object_cols = ["CryoSleep", "VIP", "Destination", "HomePlanet"]

label_X_train = X_train.copy()
label_X_valid = X_valid.copy()
final_X = X2.copy()

label_X_train.drop(["Cabin", "PassengerId"], axis = 1, inplace = True)
label_X_valid.drop(["Cabin", "PassengerId"],axis = 1, inplace = True)
final_X.drop(["Cabin", "PassengerId"],axis = 1, inplace = True)

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])
final_X[object_cols] = ordinal_encoder.fit_transform(X2[object_cols])

# Imputing the numerical columns
my_imputer = SimpleImputer()
Mid_X_train = pd.DataFrame(my_imputer.fit_transform(label_X_train))
Mid_X_valid = pd.DataFrame(my_imputer.transform(label_X_valid))
Mid_final = pd.DataFrame(my_imputer.transform(final_X))

# Imputation removed column names; put them back
Mid_X_train.columns = label_X_train.columns
Mid_X_valid.columns = label_X_valid.columns
Mid_final.columns = final_X.columns


In [13]:
model = RandomForestClassifier(random_state=1)
model.fit(Mid_X_train, y_train)

RandomForestClassifier(random_state=1)

In [14]:
# Get validation predictions and MAE
preds_valid = model.predict(Mid_X_valid)
preds_valid
y_valid

accuracy_score(y_valid, preds_valid)

0.7746090156393745

In [15]:
train_final = pd.concat([Mid_X_valid, Mid_X_train])
y_final = pd.concat([y_valid, y_train])
model.predict(train_final)
y_final.isna().sum()

model_final = RandomForestClassifier(random_state = 1)
model_final.fit(train_final, y_final)
preds = model_final.predict(Mid_final)

In [16]:
output = pd.DataFrame({"PassengerId": ftest.PassengerId, "Transported": preds})
#output["PassengerId"] = ftest["PassengerId"]

output.to_csv('submission2.csv', index = False)

Your most recent submission scored 0.78185, which is an improvement of your previous score of 0.78092. Great job!

2035 on leaderboard :(